# Elementary Neural Networks

In the introduction notebook we clarified what precisely we mean by a neural network and some key terms such as *node, neuron, activiation function*. In this notebook we will begin to build more complicated and useful neural networks. The first neural network is a historical landmark example which provided the basis for not only a useful network topology, but also a working mathematical of memory: The Hopfield Network. We will then cover the multilayer perceptron in more detail. The key learning outcomes of this notebook are: understanding how neural networks work in practice; developing a working model of memory; understanding the blueprint model for modern neural networks.

## Hopfield Networks: Ascociative Memory

The Hopfield network was a landmark development in biology and Deep Learning: for the first time it gave a working model of asscociative (non-addressable) memory as well as a generic method to train a neural network to classify objects which was, in some sense, provably reliable. The network can be formulated in a very biological minded fashion which is helpful because it allows us to not only develop useful comptutational tools, but also draw biological insight.

The task the network aims to solve is: "Given a set of classifiable input data, map the data to their classification labels". We will assume the data is encoded in a vector $v \in \mathbb{R}^n$ and the classification labels in a vector $u \in \mathbb{R}^d$. 

### Learning
Now, we know that the brain uses the Hebbian rule to learn which is often colloquially summarised as "neurones that fire togther, wire together". What this means is that neurones with strongly correlated activity patterns will have high connection weights. What does this mean for us? Well, we definitely want our network to identify the labelled data and each labels must be strongly correlated with itself. We first define a neuron to be "active" if it exceeds a certain threshold $\theta$ and "inactive" if it does not. We can decide the $\theta$ based on the data. For an activation function $f$ we arrive at:

$$ s = \text{sign}(f(v) > \theta)$$

This encodes our active neurons in state 1 and inactive neurons in state -1. Essentially, this is a very primitive statement that a neuron is firing or not. The Hebb rule then dictates for a single label $v_i$ we should examine the autocorrelation of the unit activations $s(v) s(v)^T$. Fortunately, this is a matrix! If we assume that all the data are independent then the natural thing to do is to sum them all up! Thus, we arrive at:

$$ W = \frac{1}{|\text{data}|}\sum_{i=1}^{|\text{data}|} s(v_i) s(v_i)^T $$

In [378]:
delW(v) = sign.(v) .* sign.(v)'
function constructW(data)
    W = zeros(length(data[1]), length(data[1]))
    for v in data
        W .+= delW(v)
    end
    return W ./ length(data)
end

constructW (generic function with 1 method)

### Training

We now would like to train our network. For the sake of simplicity we will create some "dummy" pre-activated patterns for the network to learn. We will assume that the patterns are of length 784 and that the neurones are activated with probability 1/2. We will create 10 patterns and hash them in a dictionary with a string class label. Finally, we will train our network using the ``constructW`` function defined 

In [423]:
using Random
data = [sign.(rand([-0.2, 0.8], 784)) for i in 1:10]
labels = [randstring(10) for i in 1:10]
hash = Dict(zip(data, labels))
trained = constructW(data)

784×784 Matrix{Float64}:
  1.0   0.4  -0.2   0.0   0.2   0.6  …   0.4   0.4   0.0   0.0   0.2   0.0
  0.4   1.0  -0.4  -0.2   0.4   0.4      0.2   0.2   0.2   0.2  -0.4   0.6
 -0.2  -0.4   1.0   0.4   0.2  -0.2      0.0   0.0  -0.4   0.0   0.2  -0.4
  0.0  -0.2   0.4   1.0   0.4   0.0     -0.2   0.2  -0.2  -0.2   0.4  -0.6
  0.2   0.4   0.2   0.4   1.0   0.2      0.0   0.0  -0.4   0.0   0.2   0.0
  0.6   0.4  -0.2   0.0   0.2   1.0  …   0.0   0.8   0.0   0.4   0.2   0.0
  0.0   0.2   0.4   0.2   0.4   0.0      0.6   0.2  -0.2   0.2   0.0  -0.2
  0.0  -0.2  -0.4   0.2   0.4   0.0     -0.2  -0.2  -0.2  -0.2   0.4  -0.2
  0.6   0.0  -0.2  -0.4  -0.2   0.2      0.4   0.0   0.4  -0.4   0.2   0.0
 -0.2   0.0  -0.2   0.0  -0.2   0.2      0.4   0.4   0.0   0.4   0.2  -0.4
 -0.8  -0.6   0.4   0.2  -0.4  -0.4  …  -0.6  -0.2   0.2  -0.2   0.0  -0.2
 -0.2   0.0  -0.2  -0.8  -0.6  -0.2      0.4   0.0   0.0   0.4  -0.6   0.4
 -0.2  -0.4   0.2   0.0  -0.6   0.2      0.0   0.4   0.0   0.4   0.2  -0.4


### Classification

We now have trained our first neural network! How do we use it? It's actually quite simple - we query the network by feeding it an activated pattern. The network outputs the matrix products of its activated inputs $ u = f(W * v)$. If we consider each column a neurone then this product is analagous to the response of each of the neurones to each of the other neurones based on their dendritic connections $W$: we are finding the response of neurone $i$ by summing the activation responses from each other neurone $j$ and adding it through its weighted connection $W_{ij}$ to neurone $i$. We now repeat this process using the new vector as input into the matrix until the network reaches a stable equilibrium.

In [424]:
function class_predict(W, hash, v; M = 100)
    u = deepcopy(v)
    
    for i = 1:M
        u .= sign.(W * u)
    end 
    
    if haskey(hash, u)
        return hash[u]
    else
        return "unclassified"
    end
end

class_predict (generic function with 1 method)

We should understand a little bit more about what has happened in training. While it seems quite simple, and intuitive, some subtilities are happening behind the scenes. What we have done is assumed that the energy landscape is connected in correlations i.e. vectors that look similiar to each other are close to each in the vector space under some metrics. We have then started with a flat energy landscape and constructively added bumps into it: we made local stable points for each $v_i$ through $v_i v_i^T$. These traditionally have been likened to the spin glass states from physics. What does this mean for us? If we are correct in our assumption that similar vectors are in some sense close to each other then we have created an energy well near our class labels. If we want to classify a new point we can rely on dynamical systems theory and simply iterate our input vector recursively until it reaches a stable point. The stable point it reachs will be one of the class labels (and hopefully the correct one!). Let's try it out on our network:

In [426]:
[(class_predict(trained, hash, i), hash[i] == class_predict(trained, hash, i)) for i in data]

10-element Vector{Tuple{String, Bool}}:
 ("sXq5uwQfNr", 1)
 ("vitoZhwsD7", 1)
 ("Fozh8d1Iy1", 1)
 ("j3L4QhUUAb", 1)
 ("io7wHoxnD7", 1)
 ("g5lOagIiM2", 1)
 ("fRqG4g4tNk", 1)
 ("r7RHJlJ1t8", 1)
 ("ZaBCP4v2WL", 1)
 ("Fvf6Tpo0yZ", 1)

We did well! Our neural network can identify a pattern of data! The interpretation that we can make about the brain is that it uses neural networks to create *content addressable* memory. That is to say our brains, under this model, are clustering things that have similar correlational properties together to store them. This is different to a computer which gives a pointer to a linear vector of data.

### Data Corruption

Let's now do some experiments to show off some of the remarkable properties of the Hopfield-Tank network. First, we will corrupt some input data by adding some random noise to it. This might be akin to blurring an image or adding some white noise to a sound file. We will corrupt our data by running through the data vector and deciding to flip the sign of our data entry and assign it a random number at that location, or leave it untouched.

In [444]:
corrupt(c) = map(x -> (rand()>0.5) ? x : -sign(x) * rand(), c)
for i in data
    pred = class_predict(trained, hash, corrupt(i))
    cl = hash[i]
    println((pred, cl, pred == cl))
end

("sXq5uwQfNr", "sXq5uwQfNr", true)
("vitoZhwsD7", "vitoZhwsD7", true)
("Fozh8d1Iy1", "Fozh8d1Iy1", true)
("j3L4QhUUAb", "j3L4QhUUAb", true)
("io7wHoxnD7", "io7wHoxnD7", true)
("g5lOagIiM2", "g5lOagIiM2", true)
("fRqG4g4tNk", "fRqG4g4tNk", true)
("r7RHJlJ1t8", "r7RHJlJ1t8", true)
("ZaBCP4v2WL", "ZaBCP4v2WL", true)
("Fvf6Tpo0yZ", "Fvf6Tpo0yZ", true)


That works better than one might expect on some simply corrupted data. We are able to get correct recall on 90% of the dataset with over half of the data being entirely corrupted. You might like to have a think about what this means for cortical operations. I would encourage you to examine various different ways of corrupting data to get a flavour for how this is working. 

### Network Corruption
Now let's look at corrupting the network itself by deleting random nodes.

In [452]:
h(x) = x .* (x > 0)
deletion_fraction = 0.5
stroke_trained = trained .* h.(rand(size(trained)...) .- deletion_fraction)
for i in data
    pred = class_predict(stroke_trained, hash, corrupt(i))
    cl = hash[i]
    println((pred, cl, pred == cl))
end

("sXq5uwQfNr", "sXq5uwQfNr", true)
("vitoZhwsD7", "vitoZhwsD7", true)
("Fozh8d1Iy1", "Fozh8d1Iy1", true)
("j3L4QhUUAb", "j3L4QhUUAb", true)
("io7wHoxnD7", "io7wHoxnD7", true)
("g5lOagIiM2", "g5lOagIiM2", true)
("fRqG4g4tNk", "fRqG4g4tNk", true)
("r7RHJlJ1t8", "r7RHJlJ1t8", true)
("ZaBCP4v2WL", "ZaBCP4v2WL", true)
("Fvf6Tpo0yZ", "Fvf6Tpo0yZ", true)


This is even more amazing than corrupting the data. We have taken our memory storage device (and classifier) and deleted half of it and it still works! If we were to delete sometimes even a single pointer in computer memory we can corrupt the entire drive. This implies that the content addressable neural network is *robust*. It can withstand modification. Think about what this implies for strokes and other brain damages and the biological advantages it confers over computer memory.

### Catastrophic Forgetting

This all seems to good to be true - why don't we just apply this method to all new labelled data in some centralised location? Then we have a robust storage of that data with good recall! Unfortunately, our network has limits. After a certain number of vectors have been encoded there is simply not enough informational space in the weights matrix to store them and the network begins to forget; quite catastrophically in fact. Let's try and add too much data.

In [457]:
new_data = [rand([-0.2, 0.8], 784) for i in 1:100]
new_labels = [randstring(10) for i in 1:100]
new_trained = (length(data) * trained .+ length(new_data) .* constructW(new_data) ) ./ (length(data) .+ length(new_data))

# We need to merge the classification keys
new_hash = merge(hash, Dict(zip(map(x -> sign.(x), new_data), new_labels)))

# Try and recall our original data
for i in data
    pred = class_predict(new_trained, hash, i)
    cl = hash[i]
    println((pred, cl, pred == cl))
end

("unclassified", "sXq5uwQfNr", false)
("unclassified", "vitoZhwsD7", false)
("unclassified", "Fozh8d1Iy1", false)
("unclassified", "j3L4QhUUAb", false)
("io7wHoxnD7", "io7wHoxnD7", true)
("unclassified", "g5lOagIiM2", false)
("unclassified", "fRqG4g4tNk", false)
("r7RHJlJ1t8", "r7RHJlJ1t8", true)
("unclassified", "ZaBCP4v2WL", false)
("Fvf6Tpo0yZ", "Fvf6Tpo0yZ", true)


While dissapointing, this leads us to yet another biological insight: why we forget. We are continuially learning and relearning things. We store these in some content addressable format and so it makes sense under this model that we tend to forget things that become old and are not continually reinforced (trained).

This model is remarkable for a number of reasons: it is simple, it is performant, and (importantly) it offers key biological insights about how real brains may work. There are a number of extensions of the model which improve the storage capacity and training time e.g. [Storkey Network]().

## Multilayer Perceptrons: The Blueprint

In the previous notebook we encountered a challenge with the perceptron: it could not model the XOR gate. We now will demonstrate that we can solve our XOR problem with a multi-layered neural network. *First, try this as an exercise!*. To start off with we need to find a way to make (0,0) and (1,1) identical as this will ensure that they will map to the same point. A potential way to do this is with the matrix:

$$ W_1 = \left( \begin{matrix} 1 & -1 \\ -1 & 1 \end{matrix} \right) $$

This mapping on our data is: $(0,0) \mapsto (0,0), (1,1) \mapsto (0,0), (0,1) \mapsto (-1,1), (1,0) \mapsto (1,-1)$. 

[Image]

Drawing this out we can see that we have collapsed the two FALSE points into the same region but the points are now all collinear. To rectify this we can use a non-linear activation function $\text{relu}(x) = x(x>0)$ and apply it to the mapped points. This is fixed on the origin point and sends the other two points to the boundary of the first quadrant.

[Image]

This is now a trivial problem for a linear classifier to solve and we can choose any that will do i.e. $W = (1, 1)$.

In [554]:
relu(x) = x .* (x .> 0)
layer1(x) = relu([1 -1; -1 1] * x)
layer2(x) = [1 1] * x
xor(x) = layer2(layer1(x))
xor.([[0,0], [0,1], [1,0], [1,1]])

4-element Vector{Vector{Int64}}:
 [0]
 [1]
 [1]
 [0]

So it is nice that we can fix the XOR problem, but it seems highly unlikely that intelligent brains are operating in this strict logical fashion. In fact, our model for the Hopfield-Tank network seems to offer more realism. The Hopfield-Tank network allowed all the neurons to communicate information to one and other and is a recurrent neural network. These exist in the brain often as lateral connections within a brain region (say V1) or as bidirectional connections between brain regions. 

In fact, a more common structure is the *feed-forward* network where information comes in through a set of neurones (say the eye cells), and is transmitted forward for processing into another region (say the colliculus), and again into a further region (say the visual cortex) and so forth. We can imagine the brain as being constructed by layers of neurons. The key insight here is the layers and the informational flow and this is precisely what the XOR network is: two layers and two neurons. We can readily generalise the concept to arbitrary layers and neurons and in doing so find ourselves with another working model of the brain. This one is actually more general because we can readily add the recurrent connections found in the Hopfield network into every layer. For this reason, multi-layer perceptrons (MLP) are what we typically refer to when we say *Artificial Neural Network* (ANN). They are usually depicted like this:

[Picture]

As we add more and more layers, the network gets deeper and deeper. This is *deep learning*. It seems that it would be useful to have a computational structure that abstracts this complexity into its core definition. We probably want to change the weights and biases so it should be a mutable structure. The connections are all-to-all (for now) in every layer. So we just need to specify the dimension of each layer and the activation function asscociated with it. We can also define a useful function that actually implements the function on data.

In [66]:
mutable struct ANN
        W::Array{Array{Float64, 2}, 1}
        b::Array{Array{Float64, 1}, 1}
        f::Array{Function, 1}
        function ANN(dims::Tuple, act::Array{Function, 1})
            gen = 2:length(dims)
            W = Array{Array{Float64, 2}, 1}([rand(dims[i], dims[i-1]) for i in gen])
            b = Array{Array{Float64, 1}, 1}([rand(dims[i]) for i in gen])
            f = Array{Function, 1}([i for i in act])
            new(W, b, f)
        end
end

function feed_forward(nn::ANN, data::Vector)
    u = similar(nn.W[1] + nn.b[1])
    for i in 1:length(nn.W)
        u = nn.f[i].(nn.W[i] * u + nn.b[i])
    end
    return u
end     

feed_forward (generic function with 2 methods)

We could make this more or less complicated. Perhaps you might like to write your own structure which takes *pre-trained* weights and biases and constructs the network from them. 

## Loss Functions
Deeper networks are also more abstract and are able to detangle higher order correlations and so are typically more powerful. This is expected, they are combinatiorally more connected, but it also means they are harder to train. This is something we haven't touched on yet. With our perceptron example we were able to train with analogy to a routine that already worked in another domain. We had to hand-code or XOR example. These MLPs seem to have many more parameters so hand-tuning is not an option and linear regression isn't possible here. 

To train them we first need to realise that they are nothing more than functions (say $\vec{{y_*}} = f(\vec{x}$) from $\mathbb{R}^n$ to $\mathbb{R}^m$ and are parameterised by their weights and biases. The other thing we need is a task or an *objective*. This is provided by the *objective function* also known as a *loss function*. The function typically takes paired labelled true data, and the predictors and combines them together to create a score that defines the object. A common loss function is mean squared error defined by:

$$ L(f(\vec{x}), y(\vec{x})) = \frac{1}{N}\sum_{i=1}^N |f(\vec{x})_i - y(\vec{x})_i|^2$$.

This useful function gives a measure of similarity between two vectors in the Euclidean distance sense (often a very good proxy for any distance, coincidentally). Can you think of what the loss function for the Hopfield-Tank network might have been? To train them we want to optimise the loss function in the weights and biases over the data. To do this we can draw on tools from functional analysis and in particular: *gradient descent*. This will be the subject of the next notebook.